# ECG Split 1D-CNN Client Side

This code is the server part of ECG split 1D-CNN model for **single** client and a server.

## Import required packages

In [1]:
import os
import struct
import socket
import pickle
import time

import h5py
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD

## Define ECG dataset class

In [2]:
class ECG(Dataset):
    def __init__(self, train=True):
        if train:
            with h5py.File(os.path.join('mitdb', 'train_ecg.hdf5'), 'r') as hdf:
                self.x = hdf['x_train'][:]
                self.y = hdf['y_train'][:]
        else:
            with h5py.File(os.path.join('mitdb', 'test_ecg.hdf5'), 'r') as hdf:
                self.x = hdf['x_test'][:]
                self.y = hdf['y_test'][:]
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])

### Set batch size

In [3]:
batch_size = 32

## Make train and test dataset batch generator

In [4]:
train_dataset = ECG(train=True)
test_dataset = ECG(train=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

### Total number of batches

In [5]:
total_batch = len(train_loader)
print(total_batch)

414


## Define ECG client model
Client side has only **2 convolutional layers**.

In [6]:
class EcgClient(nn.Module):
    def __init__(self):
        super(EcgClient, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 32 x 16
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 32 * 16)
        return x

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.get_device_name(0)

'GeForce 840M'

### Set random seed

In [8]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

### Assign intial weight as same as non-split model

In [9]:
ecg_client = EcgClient()
ecg_client.to(device)

checkpoint = torch.load("init_weight.pth")
ecg_client.conv1.weight.data = checkpoint["conv1.weight"]
ecg_client.conv1.bias.data = checkpoint["conv1.bias"]
ecg_client.conv2.weight.data = checkpoint["conv2.weight"]
ecg_client.conv2.bias.data = checkpoint["conv2.bias"]

### Set other hyperparameters in the model
Hyperparameters here should be same with the server side.

In [10]:
epoch = 400
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = Adam(ecg_client.parameters(), lr=lr)

## Socket initialization

### Required socket functions

In [11]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    return recvall(sock, msglen)

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

### Set host address and port number

In [12]:
host = 'localhost'
port = 10080
max_recv = 4096

### Open the client socket

In [13]:
s = socket.socket()
s.connect((host, port))

## Real training process

In [14]:
for e in range(epoch):
    print("Epoch {} - ".format(e+1), end='')
    
    for _, batch in enumerate(train_loader):
        x, label = batch
        x, label = x.to(device), label.to(device)
        optimizer.zero_grad()
        output = ecg_client(x)
        client_output = output.clone().detach().requires_grad_(True)
        msg = {
            'client_output': client_output,
            'label': label
        }
        msg = pickle.dumps(msg)
        send_msg(s, msg)
        msg = recv_msg(s)
        client_grad = pickle.loads(msg)
        output.backward(client_grad)
        optimizer.step()
            
    with torch.no_grad():  # calculate test accuracy
        for _, batch in enumerate(test_loader):
            x, label = batch
            x, label = x.to(device), label.to(device)
            client_output = ecg_client(x)
            msg = {
                'client_output': client_output,
                'label': label
            }
            msg = pickle.dumps(msg)
            send_msg(s, msg)
    
    msg = recv_msg(s)
    train_test_status = pickle.loads(msg)
    print(train_test_status)

Epoch 1 - loss: 1.3243, acc: 59.04% / test_loss: 1.1506, test_acc: 78.38%
Epoch 2 - loss: 1.0870, acc: 83.13% / test_loss: 1.0512, test_acc: 86.10%
Epoch 3 - loss: 1.0534, acc: 85.63% / test_loss: 1.0353, test_acc: 87.08%
Epoch 4 - loss: 1.0419, acc: 86.58% / test_loss: 1.0287, test_acc: 87.86%
Epoch 5 - loss: 1.0346, acc: 87.26% / test_loss: 1.0234, test_acc: 88.27%
Epoch 6 - loss: 1.0337, acc: 87.26% / test_loss: 1.0211, test_acc: 88.37%
Epoch 7 - loss: 1.0291, acc: 87.66% / test_loss: 1.0231, test_acc: 88.38%
Epoch 8 - loss: 1.0272, acc: 87.78% / test_loss: 1.0171, test_acc: 88.84%
Epoch 9 - loss: 1.0261, acc: 87.87% / test_loss: 1.0175, test_acc: 88.74%
Epoch 10 - loss: 1.0243, acc: 88.12% / test_loss: 1.0161, test_acc: 88.89%
Epoch 11 - loss: 1.0221, acc: 88.15% / test_loss: 1.0158, test_acc: 88.89%
Epoch 12 - loss: 1.0198, acc: 88.50% / test_loss: 1.0146, test_acc: 88.90%
Epoch 13 - loss: 1.0188, acc: 88.54% / test_loss: 1.0161, test_acc: 88.84%
Epoch 14 - loss: 1.0180, acc: 88.5

Epoch 217 - loss: 0.9140, acc: 99.11% / test_loss: 0.9227, test_acc: 98.25%
Epoch 218 - loss: 0.9142, acc: 99.09% / test_loss: 0.9222, test_acc: 98.29%
Epoch 219 - loss: 0.9144, acc: 99.04% / test_loss: 0.9235, test_acc: 98.09%
Epoch 220 - loss: 0.9138, acc: 99.12% / test_loss: 0.9226, test_acc: 98.25%
Epoch 221 - loss: 0.9162, acc: 98.87% / test_loss: 0.9230, test_acc: 98.17%
Epoch 222 - loss: 0.9153, acc: 98.96% / test_loss: 0.9248, test_acc: 97.98%
Epoch 223 - loss: 0.9157, acc: 98.97% / test_loss: 0.9280, test_acc: 97.73%
Epoch 224 - loss: 0.9141, acc: 99.09% / test_loss: 0.9247, test_acc: 98.01%
Epoch 225 - loss: 0.9144, acc: 99.06% / test_loss: 0.9228, test_acc: 98.21%
Epoch 226 - loss: 0.9136, acc: 99.14% / test_loss: 0.9236, test_acc: 98.14%
Epoch 227 - loss: 0.9156, acc: 98.93% / test_loss: 0.9238, test_acc: 98.09%
Epoch 228 - loss: 0.9139, acc: 99.11% / test_loss: 0.9241, test_acc: 98.07%
Epoch 229 - loss: 0.9140, acc: 99.08% / test_loss: 0.9228, test_acc: 98.22%
Epoch 230 - 

In [15]:
print('Finished Training!')
print('Result is on the server side.')

Finished Training!
Result is on the server side.
